In [15]:
#https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00861
import pandas as pd

In [16]:
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
df = pd.read_csv('2022.csv.gz',  names = names, compression='gzip')

In [17]:
df.head()

,ID,DATE,ELEMENT,DATA_VALUE,M-FLAG,Q-FLAG,S-FLAG,OBS-TIME
0,AE000041196,20220101,TAVG,204,H,NaN,S,NaN
1,AEM00041194,20220101,TAVG,211,H,NaN,S,NaN
2,AEM00041217,20220101,TAVG,209,H,NaN,S,NaN
3,AEM00041218,20220101,TAVG,207,H,NaN,S,NaN
4,AG000060390,20220101,TAVG,121,H,NaN,S,NaN


In [18]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}
dfst = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys())).dropna(subset=['STATE'])

In [19]:
dfst.columns

Index(['ID', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME', 'GSN FLAG',
       'HCN/CRN FLAG', 'WMO ID'],
      dtype='object')

In [20]:
nyst = dfst[['ID', 'LATITUDE', 'LONGITUDE', 'STATE', 'NAME']][dfst['STATE'].str.match("NY")]

In [21]:
nydf = df[['ID', 'DATE', 'ELEMENT', 'DATA_VALUE']][df['ID'].isin(nyst['ID'])  & df['ELEMENT'].isin(['TAVG', 'PRCP' ])]

In [22]:
ny = nydf.merge(nyst, on='ID')

In [23]:
nyp = ny.pivot(index=['NAME','LATITUDE', 'LONGITUDE', 'DATE'], columns=['ELEMENT'], values=['DATA_VALUE']).reset_index()

In [25]:
nyp.columns = [c[0] if i <4 else c[1] for i, c in enumerate(nyp.columns.to_flat_index())]

In [26]:
nyp['TAVGF'] =  (nyp['TAVG'].astype(float)/10) *(9/5) + 32

In [27]:
nyp['PRCPI'] = nyp['PRCP'].astype(float)/10 * 0.039370 

In [28]:
nyp.to_csv("nydata", index=False)